In [4]:
pip install torch

In [5]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid

# Load a sample dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class SimpleGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Use data.num_node_features after data is defined
        self.conv1 = GCNConv(data.num_node_features, 4)  # 1st layer: in_dim->4
        self.conv2 = GCNConv(4, dataset.num_classes)     # 2nd layer: 4->num_classes

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))  # aggregate + activate
        x = self.conv2(x, edge_index)          # second layer
        return F.log_softmax(x, dim=1)

model = SimpleGCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop (simplified)
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Evaluate accuracy on test nodes
model.eval()
pred = model(data).argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()
print(f'GCN Test Accuracy: {acc:.2f}')

Processing...
Done!


GCN Test Accuracy: 0.67


In [6]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid

# Load a sample dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class SimpleGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Use data.num_node_features after data is defined
        self.conv1 = GCNConv(data.num_node_features, 4)  # 1st layer: in_dim->4
        self.conv2 = GCNConv(4, dataset.num_classes)     # 2nd layer: 4->num_classes

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))  # aggregate + activate
        x = self.conv2(x, edge_index)          # second layer
        return F.log_softmax(x, dim=1)

model = SimpleGCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop (simplified)
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Evaluate accuracy on test nodes
model.eval()
pred = model(data).argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()
print(f'GCN Test Accuracy: {acc:.2f}')

GCN Test Accuracy: 0.69


In [10]:
import torch
from torch_geometric.nn import GATConv
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Load a sample dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class SimpleGAT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(data.num_node_features, 64, heads=8, concat=True)  # استخدام GAT
        self.conv2 = GATConv(64*8, 64)  # طبقة GAT ثانية
        self.conv3 = GATConv(64, dataset.num_classes)
        self.dropout = 0.5

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the model
model = SimpleGAT()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Set up learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)

# Training loop
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    scheduler.step(loss)  # تحديث معدل التعلم

# Evaluate accuracy on test nodes
model.eval()
pred = model(data).argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()
print(f'GCN Test Accuracy: {acc:.2f}')


/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:1343: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  current = float(metrics)


GCN Test Accuracy: 0.78


In [11]:
import torch
from torch_geometric.nn import GATConv
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Load a sample dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class SimpleGAT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(data.num_node_features, 64, heads=8, concat=True)  # Using GAT
        self.conv2 = GATConv(64*8, 64)  # Second GAT layer
        self.conv3 = GATConv(64, dataset.num_classes)
        self.dropout = 0.5  # Dropout rate

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))  # First GAT layer with ReLU activation
        x = F.dropout(x, p=self.dropout, training=self.training)  # Apply dropout

        x = F.relu(self.conv2(x, edge_index))  # Second GAT layer with ReLU activation
        x = F.dropout(x, p=self.dropout, training=self.training)  # Apply dropout

        x = self.conv3(x, edge_index)  # Final GAT layer
        return F.log_softmax(x, dim=1)

# Initialize the model
model = SimpleGAT()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Lower learning rate

# Set up learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)  # Reduce learning rate on plateau

# Training loop
for epoch in range(200):  # Increase epochs to 200
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])  # Compute loss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update model parameters

    scheduler.step(loss)  # Update learning rate

# Evaluate accuracy on test nodes
model.eval()
pred = model(data).argmax(dim=1)  # Get predictions
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()  # Compute accuracy
print(f'GCN Test Accuracy: {acc:.2f}')


GCN Test Accuracy: 0.78
